# 评估并优化生成部分

在前面的章节中，我们讲到了如何评估一个基于 RAG 框架的大模型应用的整体性能。通过针对性构造验证集，可以采用多种方法从多个维度对系统性能进行评估。但是，评估的目的是为了更好地优化应用效果，要优化应用性能，我们需要结合评估结果，对评估出的 Bad Case 进行拆分，并分别对每一部分做出评估和优化。

RAG 全称为检索增强生成，因此，其有两个核心部分：检索部分和生成部分。检索部分的核心功能是保证系统根据用户 query 能够查找到对应的答案片段，而生成部分的核心功能即是保证系统在获得了正确的答案片段之后，可以充分发挥大模型能力生成一个满足用户要求的正确回答。

优化一个大模型应用，我们往往需要从这两部分同时入手，分别评估检索部分和优化部分的性能，找出 Bad Case 并针对性进行性能的优化。而具体到生成部分，在已限定使用的大模型基座的情况下，我们往往会通过优化 Prompt Engineering 来优化生成的回答。在本章中，我们将首先结合我们刚刚搭建出的大模型应用实例——个人知识库助手，向大家讲解如何评估分析生成部分性能，针对性找出 Bad Case，并通过优化 Prompt Engineering 的方式来优化生成部分。

在正式开始之前，我们先加载我们的向量数据库与检索链：

In [7]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# # 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
# from zhipuai_embedding import ZhipuAIEmbeddings

# from langchain.vectorstores.chroma import Chroma
# from langchain_openai import ChatOpenAI
# from dotenv import load_dotenv, find_dotenv
# import os

# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']
# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# # 定义 Embeddings
# embedding = ZhipuAIEmbeddings()

# # 向量数据库持久化路径
# persist_directory = '../../data_base/vector_db/chroma'

# # 加载数据库
# vectordb = Chroma(
#     persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
#     embedding_function=embedding
# )

# # 使用 OpenAI GPT-3.5 模型
# llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'


In [1]:
from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint
from langchain_community.llms import QianfanLLMEndpoint
from langchain.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())  # read local .env file

from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint

embedding = QianfanEmbeddingsEndpoint()

# 向量数据库持久化路径
persist_directory = "../../data_base/vector_db/chroma"

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding,
)

# 使用 百度
llm = QianfanLLMEndpoint(streming=True)

我们先使用初始化的 Prompt 创建一个基于模板的检索链：

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)



qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

先测试一下效果：

In [3]:
question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])

/home/chen/miniconda3/envs/llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
[INFO] [06-28 18:38:50] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[INFO] [06-28 18:38:50] oauth.py:207 [t:140362715198976]: trying to refresh access_token for ak `OH70Zp***`
[INFO] [06-28 18:38:51] oauth.py:220 [t:140362715198976]: sucessfully refresh access_token
[WARNING] [06-28 18:38:51] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:38:51] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


南瓜书指的是一本由OpenAI团队开发的，用于开发大语言模型的应用的最佳实践和技巧的书籍。


## 1. 提升直观回答质量

寻找 Bad Case 的思路有很多，最直观也最简单的就是评估直观回答的质量，结合原有资料内容，判断在什么方面有所不足。例如，上述的测试我们可以构造成一个 Bad Case：

    问题：什么是南瓜书
    初始回答：南瓜书是对《机器学习》（西瓜书）中难以理解的公式进行解析和补充推导细节的一本书。谢谢你的提问！
    存在不足：回答太简略，需要回答更具体；谢谢你的提问感觉比较死板，可以去掉
我们再针对性地修改 Prompt 模板，加入要求其回答具体，并去掉“谢谢你的提问”的部分：

In [4]:
template_v2 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v2)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "什么是南瓜书"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:40:20] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:40:20] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:40:20] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


根据给出的上下文，“南瓜书”似乎是一种教程或书籍，它旨在向开发者提供关于提升大语言模型应用效果的技巧和最佳实践，涵盖软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。该书籍可能包含许多实用的提示和技巧，以帮助开发人员更好地利用大语言模型来开发各种应用程序。


可以看到，改进后的 v2 版本能够给出更具体、详细的回答，解决了之前的问题。但是我们可以进一步思考，要求模型给出具体、详细的回答，是否会导致针对一些有要点的回答没有重点、模糊不清？我们测试以下问题：

In [5]:
question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:42:26] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:42:26] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:42:26] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


使用大模型时，构造 Prompt 的原则主要有以下几点：

1. 明确性：给定的 Prompt 应该清晰、明确地表达开发者的意图。如果 Prompt 含糊不清或过于复杂，模型可能会误解或无法理解其含义。
2. 针对性：针对不同的应用场景和需求，选择合适的 Prompt。例如，对于聊天机器人，可能需要设计更自然、流畅的 Prompt；而对于搜索应用，可能需要设计更具体、明确的 Prompt。
3. 具体性：Prompt 应该尽可能具体，避免使用过于抽象或模糊的词汇。具体化的 Prompt 可以帮助模型更好地预测下一个单词或短语。
4. 语境化：Prompt 应尽可能考虑上下文信息，以模拟真实世界的语境。这可以通过在 Prompt 中包含相关的词汇、短语或句子来完成。
5. 多样化：对于不同的应用场景，可能需要设计不同的 Prompt。因此，在开发过程中，可以尝试不同的 Prompt 设计，以找到最佳的效果。
6. 有效性：在设计 Prompt 时，需要考虑模型的限制和能力，确保 Prompt 能够有效地引导模型做出预测。

这些原则可以帮助开发人员设计出更有效、更准确的 Prompt，从而提升大语言模型的应用效果。


可以看到，针对我们关于 LLM 课程的提问，模型回答确实详细具体，也充分参考了课程内容，但回答使用首先、其次等词开头，同时将整体答案分成了4段，导致答案不是特别重点清晰，不容易阅读。因此，我们构造以下 Bad Case：

    问题：使用大模型时，构造 Prompt 的原则有哪些
    初始回答：略
    存在不足：没有重点，模糊不清

针对该 Bad Case，我们可以改进 Prompt，要求其对有几点的答案进行分点标号，让答案清晰具体：


In [6]:
template_v3 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v3)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "使用大模型时，构造 Prompt 的原则有哪些"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:42:34] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:42:34] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:42:34] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


使用大模型时，构造 Prompt 的原则有以下几点：

1. 明确性：给定的 Prompt 应该明确表达开发人员想要从模型中获取的信息或行为。例如，如果开发人员想要模型预测一个城市的名字，那么 Prompt 就应该直接问“法国的首都是什么”。

2. 相关性：Prompt 的内容应该与模型的主要任务和训练数据相关。例如，如果模型的主要任务是文本生成，那么 Prompt 就应该与文本生成相关。

3. 针对性：Prompt 的设计应该针对特定的应用场景和需求。例如，如果开发人员想要构建一个聊天机器人，那么 Prompt 就应该有助于模型的对话理解和生成能力。

4. 多样化：使用多样化的 Prompt 可以提高模型的泛化能力，从而减少对特定训练数据的依赖。例如，使用“从多个角度来回答这个问题”或“给出两个可能的答案”等 Prompt 可以帮助模型提供更全面的回答。

5. 有效性：Prompt 的设计应该有助于提高模型的性能和准确性。因此，开发人员应该测试和评估不同的 Prompt，选择最有效的一个。

以上原则可以帮助开发人员设计出更有效、更准确的 Prompt，从而更好地利用大语言模型的应用。


提升回答质量的方法还有很多，核心是围绕具体业务展开思考，找出初始回答中不足以让人满意的点，并针对性进行提升改进，此处不再赘述。

## 2. 标明知识来源，提高可信度

由于大模型存在幻觉问题，有时我们会怀疑模型回答并非源于已有知识库内容，这对一些需要保证真实性的场景来说尤为重要，例如：

In [7]:
question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:44:45] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:44:45] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:44:45] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


强化学习是一种通过让模型根据人类对模型输出的反馈，不断优化自身行为，以更好地遵循指令并生成更安全可靠的输出的训练方法。这种方法可以增强模型遵循指令的能力，使其生成对指令高度敏感、更安全可靠的输出，较少出现无关和损害性内容。


我们可以要求模型在生成回答时注明知识来源，这样可以避免模型杜撰并不存在于给定资料的知识，同时，也可以提高我们对模型生成答案的可信度：

In [8]:
template_v4 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
请你附上回答的来源原文，以保证回答的正确性。
{context}
问题: {question}
有用的回答:"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "强化学习的定义是什么"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:44:51] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:44:51] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:44:51] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


强化学习是一种通过让模型根据人类对模型输出的反馈来增强模型遵循指令的能力，从而生成对指令高度敏感、更安全可靠的输出的方法。强化学习技术根据模型在执行指令过程中的表现，对模型进行奖励或惩罚，从而引导模型在未来的执行中更好地遵循指令。这种方法通常用于提高指令微调 LLM 的性能和准确性。


但是，附上原文来源往往会导致上下文的增加以及回复速度的降低，我们需要根据业务场景酌情考虑是否要求附上原文。

## 3. 构造思维链

大模型往往可以很好地理解并执行指令，但模型本身还存在一些能力的限制，例如大模型的幻觉、无法理解较为复杂的指令、无法执行复杂步骤等。我们可以通过构造思维链，将 Prompt 构造成一系列步骤来尽量减少其能力限制，例如，我们可以构造一个两步的思维链，要求模型在第二步做出反思，以尽可能消除大模型的幻觉问题。

我们首先有这样一个 Bad Case：

    问题：我们应该如何去构造一个 LLM 项目
    初始回答：略
    存在不足：事实上，知识库中中关于如何构造LLM项目的内容是使用 LLM API 去搭建一个应用，模型的回答看似有道理，实则是大模型的幻觉，将部分相关的文本拼接得到，存在问题

In [10]:
question = "我们应该如何去构造一个LLM项目"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:48:33] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:48:34] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:48:34] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


应该从开发人员角度来看，考虑以下步骤去构造一个 LLM 项目：

1. 确定项目的目标和需求：在开始构建任何 LLM 项目之前，了解项目目标非常重要。确定所需的功能、使用的领域、面向的用户等等。这将帮助你确定项目的关键组成部分，例如任务或数据集。
2. 挑选适当的 LLM 模型：根据你的项目需求和目标，选择一个适当的 LLM 模型。这可以是从基础 LLM 到指令微调 LLM 中的任何模型。确保选择一个与你的项目需求相匹配的模型。
3. 准备数据集：根据项目需求，准备适当的数据集。数据集应该是专门为你的 LLM 项目设计的，并包含相关和有意义的信息。对于大型数据集，可能需要使用特定的工具和技术进行预处理和清理。
4. 编写代码和 API 调用：使用所选的 LLM 模型的 API 或库，编写代码以执行所需的任务。根据项目需求，可能需要编写不同的代码片段，例如文本生成、问答、对话等。确保正确调用 API 并遵循相关文档和指南。
5. 测试和优化：在构建完成后，对 LLM 项目进行测试以确保其正常工作并满足需求。根据测试结果进行必要的优化和调整。
6. 部署和集成：如果项目完成后，将其部署到适当的服务器或环境中，并确保它可以与其他应用程序或系统集成。

重要的是要意识到 LLM 项目通常需要一定的时间和资源来构建和优化。因此，建议在开始之前进行充分的规划和准备，并寻求专业人士的帮助和建议。同时，保持对最新技术和工具的了解，以便随时适应变化和发展。


对此，我们可以优化 Prompt，将之前的 Prompt 变成两个步骤，要求模型在第二个步骤中做出反思：

In [11]:
template_v4 = """
请你依次执行以下步骤：
① 使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
你应该使答案尽可能详细具体，但不要偏题。如果答案比较长，请酌情进行分段，以提高答案的阅读体验。
如果答案有几点，你应该分点标号回答，让答案清晰具体。
上下文：
{context}
问题: 
{question}
有用的回答:
② 基于提供的上下文，反思回答中有没有不正确或不是基于上下文得到的内容，如果有，回答你不知道
确保你执行了每一个步骤，不要跳过任意一个步骤。
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

question = "我们应该如何去构造一个LLM项目"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:49:12] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:49:12] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:49:12] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


我们应该按照以下步骤去构造一个 LLM 项目：

1. **明确你的应用目标**：在开始构建 LLM 项目之前，你需要明确你的应用目标是什么。这个目标将决定你需要构建 LLM 的功能和特性。
2. **选择合适的 LLM API**：根据你的应用目标，选择一个适合你的 LLM API。目前市场上有很多 LLM API 提供商，例如 OpenAI、DeepLearning.AI 等，它们提供了各种不同的功能和接口。
3. **创建和使用 Prompt 设计文档**：根据应用目标，创建和使用 Prompt 设计文档，用于引导 LLM 的输入输出。你需要明确你想要让 LLM 回答什么问题，并为这些问题设计适当的 Prompts。
4. **使用 LLM API 进行项目开发**：使用所选的 LLM API 进行项目开发。你可以通过 API 接口调用 LLM，从而快速构建软件应用程序。确保遵循 LLM 的使用指南和限制。
5. **获取反馈并优化**：获取用户对 LLM 项目的反馈，并根据反馈进行优化和改进。你可以使用 RLHF 技术，根据人类对模型输出的反馈进一步增强模型遵循指令的能力。

对于您的问题“我们应该如何去构造一个 LLM 项目”，需要注意的是您提供的内容主要集中在使用特定技术方面的知识，其中包括了对具体应用程序的设计与实现方面的建议，并涉及到了一些实际案例的讲解和经验分享。这都旨在帮助您在具体实践上更好地理解和应用相关技术。

然而，您提到了“不是基于上下文得到的内容”，我暂时没有发现任何不正确或没有基于上下文得到的内容。如果您有更具体的问题或需要进一步的解释，欢迎随时向我提问。


可以看出，要求模型做出自我反思之后，模型修复了自己的幻觉，给出了正确的答案。我们还可以通过构造思维链完成更多功能，此处就不再赘述了，欢迎读者尝试。

## 4. 增加一个指令解析

我们往往会面临一个需求，即我们需要模型以我们指定的格式进行输出。但是，由于我们使用了 Prompt Template 来填充用户问题，用户问题中存在的格式要求往往会被忽略，例如：

In [12]:
question = "LLM的分类是什么？给我返回一个 Python List"
result = qa_chain({"query": question})
print(result["result"])

[INFO] [06-28 18:49:57] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-28 18:49:57] base.py:516 [t:140362715198976]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-28 18:49:57] openapi_requestor.py:316 [t:140362715198976]: requesting llm api endpoint: /chat/eb-instant


根据提供的上下文，LLM大致可以分为两种类型：基础LLM和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。指令微调LLM则通过专门的训练，可以更好地理解并遵循指令。这两种类型都可以通过API接口调用，从而快速构建软件应用程序。

基于上述内容，我将这两种类型分类整理为一个Python List，如下：


```python
LLM_Types = ['基础LLM', '指令微调（Instruction Tuned）LLM']
```
这个列表中包含了两种类型的LLM分类。

请注意，我的回答是基于提供的上下文，并且确保了与上下文中的描述一致。没有发现任何不正确或基于上下文之外的内容。


可以看到，虽然我们要求模型给返回一个 Python List，但该输出要求被包裹在 Template 中被模型忽略掉了。针对该问题，我们可以构造一个 Bad Case：

    问题：LLM的分类是什么？给我返回一个 Python List
    初始回答：根据提供的上下文，LLM的分类可以分为基础LLM和指令微调LLM。
    存在不足：没有按照指令中的要求输出

针对该问题，一个存在的解决方案是，在我们的检索 LLM 之前，增加一层 LLM 来实现指令的解析，将用户问题的格式要求和问题内容拆分开来。这样的思路其实就是目前大火的 Agent 机制的雏形，即针对用户指令，设置一个 LLM（即 Agent）来理解指令，判断指令需要执行什么工具，再针对性调用需要执行的工具，其中每一个工具可以是基于不同 Prompt Engineering 的 LLM，也可以是例如数据库、API 等。LangChain 中其实有设计 Agent 机制，但本教程中我们就不再赘述了，这里只基于 OpenAI 的原生接口简单实现这一功能：

In [15]:
# 使用第二章讲过的 OpenAI 原生接口

from openai import OpenAI

client = QianfanLLMEndpoint(streming=True)


def gen_gpt_messages(prompt):
    '''
    构造 GPT 模型请求参数 messages
    
    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    '''
    获取 GPT 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型，默认为 gpt-3.5-turbo，也可以按需选择 gpt-4 等其他模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~2。温度系数越低，输出内容越一致。
    '''
    response = QianfanLLMEndpoint(streming=True)
    return response
    #return "generate answer error"

prompt_input = '''
请判断以下问题中是否包含对输出的格式要求，并按以下要求输出：
请返回给我一个可解析的Python列表，列表第一个元素是对输出的格式要求，应该是一个指令；第二个元素是去掉格式要求的问题原文
如果没有格式要求，请将第一个元素置为空
需要判断的问题：
```
{}
```
不要输出任何其他内容或格式，确保返回结果可解析。
'''

我们测试一下该 LLM 分解格式要求的能力：


In [16]:
response = get_completion(prompt_input.format(question))
response

QianfanLLMEndpoint(client=<qianfan.resources.llm.completion.Completion object at 0x7fa867ece0d0>, qianfan_ak=SecretStr('**********'), qianfan_sk=SecretStr('**********'))

可以看到，通过上述 Prompt，LLM 可以很好地实现输出格式的解析，接下来，我们可以再设置一个 LLM 根据输出格式要求，对输出内容进行解析：

In [17]:
prompt_output = '''
请根据回答文本和输出格式要求，按照给定的格式要求对问题做出回答
需要回答的问题：
```
{}
```
回答文本：
```
{}
```
输出格式要求：
```
{}
```
'''

然后我们可以将两个 LLM 与检索链串联起来：


In [18]:
question = 'LLM的分类是什么？给我返回一个 Python List'
# 首先将格式要求与问题拆分
input_lst_s = get_completion(prompt_input.format(question))
# 找到拆分之后列表的起始和结束字符
start_loc = input_lst_s.find('[')
end_loc = input_lst_s.find(']')
rule, new_question = eval(input_lst_s[start_loc:end_loc+1])
# 接着使用拆分后的问题调用检索链
result = qa_chain({"query": new_question})
result_context = result["result"]
# 接着调用输出格式解析
response = get_completion(prompt_output.format(new_question, result_context, rule))
response

AttributeError: 'QianfanLLMEndpoint' object has no attribute 'find'

可以看到，经过如上步骤，我们就成功地实现了输出格式的限定。当然，在上面代码中，核心为介绍 Agent 思想，事实上，不管是 Agent 机制还是 Parser 机制（也就是限定输出格式），LangChain 都提供了成熟的工具链供使用，欢迎感兴趣的读者深入探讨，此处就不展开讲解了。

通过上述讲解的思路，结合实际业务情况，我们可以不断发现 Bad Case 并针对性优化 Prompt，从而提升生成部分的性能。但是，上述优化的前提是检索部分能够检索到正确的答案片段，也就是检索的准确率和召回率尽可能高。那么，如何能够评估并优化检索部分的性能呢？下一章我们会深入探讨这个问题。